# DAC Contest
This reference design will help you walk through a design flow of DAC SDC 2023. This is a simplified design to help users get started on the FPGA platform and to understand the overall flow. It does not contain any object detection hardware.

If you have any questions, please post on the Slack page (link on SDC website sidebar).

### Hardware

### Software
Note:
  * You will not submit your `dac_sdc.py` file, so any changes you make to this file will not be considered during evluation.  
  * You can use both PS and PL side to do inference.

### Object Detection

Object detection will be done on images in batches:
  * You will provide a Python callback function that will perform object detection on batch of images.  This callback function wile be called many times.
  * The callback function should return the locations of all images in the batch.
  * Runtime will be recorded during your callback function.
  * Images will be loaded from SD card before each batch is run, and this does not count toward your energy usage or runtime.
  
### Notebook
Your notebook should contain 4 code cells:

1. Importing all libraries and creating your Team object.
1. Downloading the overlay, compile the code, and performany any one-time configuration.
1. Python callback function and any other Python helper functions.
1. Running object detection
1. Cleanup



## 1. Imports and Create Team

In [1]:
import sys
import os

sys.path.append(os.path.abspath("../common"))

import math
import time
from pathlib import Path
import numpy as np
from PIL import Image
from matplotlib import pyplot
import cv2
from datetime import datetime

import dac_sdc
from iris_utils import TensorRTEngine, preprocess, postprocess, artifacts2output

team_name = 'IRIS'
engine_path = 'IRIS.trt'
dac_sdc.BATCH_SIZE = 2
team = dac_sdc.Team(team_name)

**Your team directory where you can access your notebook, and any other files you submit, is available as `team.team_dir`.**

## 2. Preparing the library and model
Prepare the dependencies for the contest, including installing python packages, compiling your binaries, and linking to the notebook.

Your team is responsible to make sure the correct packages are installed. For the contest environment, use the configuration below provided by Nvidia:
- [JetPack 4.6.1](https://developer.nvidia.com/embedded/jetpack-sdk-461)
    - Ubuntu 18.04
    - CUDA 10.2
    - cuDNN 8.2.1
    - gcc 7.5.0
    - python 3.6.9
    - TensorRT 8.2.1

## 3. Python Callback Function and Helper Functions


### Pushing the picture through the pipeline
In this example, we use contiguous memory arrays for sending and receiving data via DMA.

The size of the buffer depends on the size of the input or output data.  The example images are 640x360 (same size as training and test data), and we will use `pynq.allocate` to allocate contiguous memory.

### Callback function
The callback function:
  - Will be called on each batch of images (will be called many times)
  - Is prvided with a list of tuples of (image path, RGB image)
  - It should return a dictionary with an entry for each image:
    - Key: Image name (`img_path.name`)
    - Value: Dictionary of item type and bounding box (keys: `type`, `x`, `y`, `width`, `height`)

See the code below for an example:


In [2]:
# For profiling

class AvgTimeProfiler:
    def __init__(self):
        self.n = 0
        self.v = 0.
    
    def update(self, t):
        self.v = ((self.v * self.n) + t) / (self.n + 1)
        self.n += 1
    
    @property
    def result(self):
        return self.v
    
atp1 = AvgTimeProfiler()
atp2 = AvgTimeProfiler()
atp3 = AvgTimeProfiler()
atp4 = AvgTimeProfiler()

from tqdm import tqdm
from functools import partial

In [3]:
def create_callback(engine_path):
    engine = TensorRTEngine(engine_path)
    input_shape = engine.input_shape # (1, 3, h, w)
    conf_thres = 0.25
    iou_thres = 0.7
    max_det = 200
    tbar = tqdm()
    
    def callback(inputs, use_tqdm=False):
        outputs = dict()
        for img_path, rgb_img in inputs:
            orig_img_shape = rgb_img.shape
            
            ## Pre-process ##
            if use_tqdm: t1 = time.time()
            rgb_img = preprocess(rgb_img, input_shape[2:], is_rgb=True)
            
            ## Run TRT-engine ##
            if use_tqdm: t2 = time.time()
            preds = engine.run(rgb_img)
            
            ## Post-process ##
            if use_tqdm: t3 = time.time()
            clss, boxes, masks = postprocess(
                preds, input_shape[2:], orig_img_shape,
                conf_thres=conf_thres,
                iou_thres=iou_thres,
                max_det=max_det,
            )
            
            ## Organize results ##
            if use_tqdm: t4 = time.time()
            outputs[img_path.name] = artifacts2output(clss, boxes, masks)
            
#             ## For dummy #
#             outputs[img_path.name] = [{"type": 1, "x": -1, "y": -1, "width": -1, "height": -1, "segmentation": []}]

            if use_tqdm: 
                t5 = time.time()
                atp1.update(t2 - t1)
                atp2.update(t3 - t2)
                atp3.update(t4 - t3)
                atp4.update(t5 - t4)
                tbar.set_postfix(
                    pre=atp1.result,
                    engine=atp2.result,
                    post=atp3.result,
                    art2out=atp4.result,
                )
                tbar.update(1)
        return outputs
    return callback


iris_callback = create_callback(engine_path)
print("warmup")
dummy = [(Path('hello.txt'), np.zeros((640, 640, 3), dtype=np.uint8)) for _ in range(10)]
iris_callback(dummy)
print("done")

0it [00:00, ?it/s]

warmup
done


## 4. Running Object Detection

Call the following function to run the object detection.  Extra debug output is enabled when `debug` is `True`.

In [ ]:
team.run(partial(iris_callback, use_tqdm=True), debug=False)

# team.run(iris_callback, debug=True)

2552it [11:32,  3.60it/s, art2out=0.000141, engine=0.1, post=0.113, pre=0.0123]  